**1. Problema de negócio**

Um e-commerce "X" necessita aumentar suas vendas, em prol da sua sustentabilidade. Sabendo o que ofertar e no momento certo para seus clientes, as chances de venda crescem. Dentro deste contexto, como é possível otimizar as vendas ofertando produtos que tenham forte relação com os itens que o cliente navegou?

**Atividades**:

1) Apresentar os principais indicadores:

a) total de navegações
b) número de vendas
c) faturamento no período
d) ticket médio
e) total de usuários únicos
f) taxa de conversão por usuário único

2) Construir um algoritmo para responder a seguinte questão: qual produto deve ser recomendado para um usuário que navegou pelo produto 76700399 ?

**Conjunto de dados**: captura de clickstream de um e-commerce fictício.

                                        Descrição dos atributos do conjunto de dados:

| Atributo | Descrição |
| --- | --- |
| user_id | Identificador único do usuário |
| action | Ação do usuário com o produto |
| id | Identificador único do produto |
| variant | Tamanho ou cor do produto |
| category | Categoria do produto |
| price | Preço do produto |
| timestamp| Data e horário da navegação |

**2. Importação de Bibliotecas**

In [0]:
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import glob

from pandas import read_csv, DataFrame, concat, get_dummies #, to_datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

**3. Aquisição dos Dados**

In [0]:
url_zip = "https://xxxxxxxx.xx.xxxxxx.com/public/clickstream.zip"
csv_file = "clickstream.csv"

# se o arquivo estiver local na máquina, o consome
if csv_file in glob.glob(csv_file):
    # considera o arquivo do repositório local
    sourcefile = csv_file
    # importando os dados para um dataframe Pandas
    df_raw = read_csv(sourcefile, sep=',')
# se o arquivo não estiver local na máquina, busca do repositório externo em nuvem e o consome
else:
    sourcefile = url_zip
    # importando os dados para um dataframe Pandas, considerando ser um arquivo .ZIP
    df_raw = read_csv(sourcefile, compression='zip', header=0, sep=',')

In [20]:
df_raw.head(5)

,user_id,action,id,variant,category,price,timestamp
0,912201561074350768,detail,72104958,NaN,Calçados,199.99,2019-06-21T00:00:02.102Z
1,309931560815276971,detail,70102327,NaN,Calçados,99.99,2019-06-21T00:00:04.700Z
2,553421545850145995,detail,61003149,NaN,Calçados,79.99,2019-06-21T00:00:05.855Z
3,389701561075193910,detail,70210195,NaN,Calçados,209.99,2019-06-21T00:00:06.853Z
4,865841560095238910,detail,68300190,NaN,Roupas,169.99,2019-06-21T00:00:06.832Z


**4. Definições do Dataset**

In [21]:
print('--->>> Qtd. observações (linhas):', df_raw.shape[0])
print('--->>> Qtd. variáveis (colunas):', df_raw.shape[1])
print('--->>> Definição da Matriz:', df_raw.shape, '\n')

print("--->>> Tipagem das colunas do dataset:")
print(df_raw.dtypes,'\n')

--->>> Qtd. observações (linhas): 617886
--->>> Qtd. variáveis (colunas): 7
--->>> Definição da Matriz: (617886, 7) 

--->>> Tipagem das colunas do dataset:
user_id        int64
action        object
id            object
variant       object
category      object
price        float64
timestamp     object
dtype: object 



In [0]:
# cópia de segurança dos dados RAW. df_raw é a fonte original, e não deve ser alterada a sua integridade.

df_adjust_01 = df_raw

In [0]:
# Ajuste da tipagem do user_id para string

df_adjust_01["user_id"] = df_adjust_01["user_id"].astype('str')

**5. Análise Exploratória**

- **5.1. Medidas de Posição - Sumarização dos Dados**

In [24]:
df_adjust_01.describe()

,price
count,613593.000000
mean,107.276364
std,60.771009
min,1.299000
25%,69.990000
50%,99.990000
75%,149.990000
max,1299.990000


- - **Valores Únicos por Coluna do DataFrame**

In [25]:
for column in df_adjust_01.columns:
    if column != "user_id" and column != "id" and column != "price" and column != "timestamp":
        print("Valores distintos da coluna:", column)
        print(df_adjust_01[column].unique())
        print("\n")

Valores distintos da coluna: action
['detail' 'cart' 'checkout' 'purchase']


Valores distintos da coluna: variant
[nan '97' '38' '92' '96' '36' '33' '95' '41' '34' '35' '44' '40' '42' '37'
 '39' '17' '99' '20' '18' '32' '27' '43' '29' '94' '46' 'or' '24' '26'
 'ck' '21' '25' '31' '19' '28' '23' '30' 'og' 'to' '02' '48' '88' '87' 'g'
 'ua' 'l' '-c' 'pi' '22' 'nh' '90' '93' '03' '-e' '89' 'er' '52' 'on' '-f'
 'y' '16' '05' '91' 'ev' '04' 'zu' '12' '15' 'lo' 'al' '45' '14' '13' '06'
 '50' '98']


Valores distintos da coluna: category
['Calçados' 'Roupas' 'Sapatos' 'Sandálias' 'Meias' 'Tênis'
 'Casacos e Jaquetas' 'Botas' 'Tamancos' 'Acessórios' 'Sapatilhas'
 'Bolsas' 'Camisas' nan 'Chinelos' 'Blusas' 'Lingeries'
 'Calças e Calças Jeans' 'Moda Íntima' 'Vestidos' 'Bermudas e Shorts'
 'Sapatênis' 'Macacões' 'Camisetas' 'Agasalhos e Conjuntos' 'Cuecas'
 'Cuidados' 'Colete' 'Saias' 'Cintos' 'Carteiras' 'Mochilas' 'Chuteiras'
 'Escolar' 'Moda Praia' 'Lingerie' 'Underwear']




- - **A) Quantidade Total de Navegação**

In [26]:
qtdusers = len(df_adjust_01)
print("Quantidade Total de Navegação:", qtdusers)

Quantidade Total de Navegação: 617886


- - **B) Quantidade Total de Vendas**

In [27]:
qtdpurchase = len(df_adjust_01[df_adjust_01["action"] == "purchase"])
print("Quantidade Total de Vendas:", qtdpurchase)

Quantidade Total de Vendas: 4771


- - **C) Faturamento no Período**

In [28]:
billing = df_adjust_01[df_adjust_01["action"] == "purchase"]
totalbilling = round(billing["price"].sum(), 2)

print("Faturamento no Período", totalbilling)

Faturamento no Período 467897.2


- - **D) Ticket Médio**

In [29]:
print("Ticket Médio:", round( (totalbilling / qtdpurchase), 2) )

Ticket Médio: 98.07


- - **E) Quantidade Total de Usuários Únicos**

In [30]:
qtdidentityusers = len(df_adjust_01["user_id"].unique())
print("Quantidade Total de Usuários Únicos:", qtdidentityusers)

Quantidade Total de Usuários Únicos: 163211


- - **F) Taxa de Conversão por Usuário Único**

    % de visitantes distintos do e-commerce que efetuaram uma compra.

In [31]:
conversionrate = round( ((qtdpurchase / qtdidentityusers) * 100) , 2)
print("Taxa de Conversão por Usuário Único:", conversionrate, "%")

Taxa de Conversão por Usuário Único: 2.92 %


**5. Modelo de Recomendação de Produtos**

- **Técnica selecionada**: análise de ASSOCIAÇÃO DE MARKET BASKET ao nível de transação ("Cliente que comprou este produto, também comprou este.").

- **Motivos que levaram à escolha da técnica para o problema em questão**
      -> simplicidade;
      -> conjunto de dados atual não permite o entendimento detalhado dos produtos e a sua relação com os cliente (se houver muitos atributos que especificam a característica dos itens, uma abordagem de recomendação baseada em FILTRO DE CONTEÚDO é a melhor opção);
      -> conjunto de dados atual não permite o entendimento detalhado dos clientes e a sua relação com os itens (se houver muitos atributos que especificam a característica dos clientes associados aos itens, uma abordagem de recomendação baseada em FILTRO COLABORATIVO é a melhor opção);

- **Vantagens**
      -> força da relação entre cada um dos produtos e todos os outros produtos ofertados;
      -> identificar pares com afinidade forte;
      -> criar uma oferta personalizada para os clientes que tem apenas um dos produtos dos pares fortemente associados.

- **Medidas p/ Mensurar a Associação**

      -> Support (Suporte): frequência de cada item contido no conjunto de dados.
      - Fórmula: 𝑆𝑢𝑝𝑝𝑜𝑟𝑡(𝑋) = (Qtd. 𝑂𝑐𝑜𝑟𝑟ê𝑛𝑐𝑖𝑎𝑠(𝑋) / 𝑇𝑜𝑡𝑎𝑙 𝑑𝑒 𝑟𝑒𝑔𝑖𝑠𝑡𝑟𝑜) X 100

      -> Confidence (Confiança): probabilidade de ocorrência do item Y quando o item X aparece.
      - Fórmula: 𝐶𝑜𝑛𝑓𝑖𝑑𝑒𝑛𝑐𝑒 𝑋 → 𝑌 = 𝑆𝑢𝑝𝑝𝑜𝑟𝑡(𝑋, 𝑌) / 𝑆𝑢𝑝𝑝𝑜𝑟𝑡(𝑋)

      -> Lift: probabilidade de um item Y aparecer quando X ocorre, em relação a popularidade de Y (considerar itens > 1).
      - Fórmula: 𝐿𝑖𝑓𝑡(𝑋 → 𝑌) = 𝐶𝑜𝑛𝑓𝑖𝑑𝑒𝑛𝑐𝑒 (𝑋 → 𝑌) / 𝑆𝑢𝑝𝑝𝑜𝑟𝑡 (𝑌)

- **Algoritmo**: Apriori
      Consegue identificar os itens que aparecem com maior frequência dentro de conjuntos de itens, considerando as possibilidades de  combinações existentes, respeitando o **suporte mínimo** definido (limiar). Se um item não for frequente, todas as suas demais combinações serão desconsideradas.
    
    Etapas:
    
    1º) Inicializa com conjuntos de itens único;
    
    2º) Obtêm o valor de suporte para cada conjunto de itens;
    
    3º) Remove os conjuntos de itens que estão abaixo do especificado para suporte mínimo;
    
    4º) Para os itens restantes, gera todas as configurações possíveis de conjunto de itens;
    
    5º) Repete as etapas de 2 a 4 até que não existam mais novos conjuntos de itens;

In [32]:
product = "76700399"

# selecionado as colunas que irão ser trabalhadas
df_adjust_02 = df_adjust_01[["user_id", "id"]]

# selecionando as pessoas que compraram o produto indicado
df_adjust_03 = df_adjust_02[df_adjust_02["id"] == product]
df_adjust_03 = df_adjust_03["user_id"]

# remove registros duplicados
df_adjust_03.drop_duplicates(inplace=True)

# exportando id's de usuário que compraram o produto indicado
indexes = df_adjust_03.values.tolist()

# criando um dataframe com os id's de usuários que compraram o produto indicado
df_adjust_04 = df_adjust_02[df_adjust_02["user_id"].isin(indexes)]
df_adjust_04.drop_duplicates(inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/base.py:1512: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return self._update_inplace(result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
# exibindo a lista de navegação de cada usuário desde que este tenha qualquer ação sobre o produto indicado
df_adjust_04.head(5)

,user_id,id
8221,448681561080412476,76700399
8237,448681561080412476,7670039912
10580,450201561082120947,76700399
10601,450201561082120947,7670039912
17290,577241561089106675,76700399


In [34]:
print("Quantidade de registros", len(df_adjust_04))

Quantidade de registros 603


In [35]:
# criando one hot enconding
columns = ["id"] # coluna do produto

df_columns = df_adjust_04.columns

for column in columns:
  for df_column in df_columns:
    if column == df_column:
      # Get one hot encoding of columns B
      one_hot = get_dummies(data = df_adjust_04[column], columns=[column])
      # Drop column B as it is now encoded
      df_adjust_04.drop(column, axis = 1, inplace = True)
      # Join the encoded df
      df_adjust_04 = df_adjust_04.join(one_hot)

# agrupa registros pelo user_id, fazendo um merge das colunas "duplicadas"
df_adjust_05 = df_adjust_04.groupby("user_id", axis = 0).sum()

# listando o dataset agregado
df_adjust_05.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,53701022,5370102212,55005653,55005655,6000657312,60006665,60006739,6000675704,6000679161,60105408,60105732,60105837,6010583712,60105848,6010584811,60105881,6010588112,60105902,60305259,60305263,60305466,6030546602,6030546616,60305559,6030555961,6030564002,6030564021,60305698,60305711,6030571112,60305712,60305713,6030571312,6060435413,60604451,60604456,60604469,6060446912,6060448266,60604483,...,76700490,76700510,7670051003,76700512,76700513,7670051303,76700514,7670051410,76700515,7670051502,76700516,7670051602,76700517,7670051703,7670051712,76700518,7670051812,76700519,7670051902,7670051903,7670051912,7670051975,76700521,76700522,7670052202,7670052275,76700523,7670052302,7670052303,80402384,80502476,8050247685,8070091205,82203647,82204057,8220405712,83201291,8320129134,83201581,8320158103
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100301560214469309,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
101801561326641304,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
103515069369234156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
117081561483306117,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
120201561320927468,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
print("Shape:", df_adjust_05.shape)

Shape: (101, 364)


In [37]:
'''
# Transformando o DataFrame em listas contidas dentro de uma "grande" lista
len_lines = len(df_adjust_05)
len_columns = len(df_adjust_05.columns)

transactions = []  
for i in range(0, len_lines):
    transactions.append([str(df_adjust_05.values[i,j]) for j in range(0, len_columns)])
'''

'\n# Transformando o DataFrame em listas contidas dentro de uma "grande" lista\nlen_lines = len(df_adjust_05)\nlen_columns = len(df_adjust_05.columns)\n\ntransactions = []  \nfor i in range(0, len_lines):\n    transactions.append([str(df_adjust_05.values[i,j]) for j in range(0, len_columns)])\n'

In [0]:
# Aplicando algoritmo Apriori
frequent_itemsets = apriori(df_adjust_05, min_support=0.05, use_colnames=True)

In [39]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(76700399),(7670039902),1.000000,0.108911,0.108911,0.108911,1.0,0.0,1.0
1,(7670039902),(76700399),0.108911,1.000000,0.108911,1.000000,1.0,0.0,inf
2,(76700399),(7670039905),1.000000,0.138614,0.138614,0.138614,1.0,0.0,1.0
3,(7670039905),(76700399),0.138614,1.000000,0.138614,1.000000,1.0,0.0,inf
4,(76700399),(7670039910),1.000000,0.148515,0.148515,0.148515,1.0,0.0,1.0


In [40]:
support = 0.08 # 8%
confidence = 0.7 # 75%
lift = 3

rules[ (rules['lift'] >= lift) & (rules['confidence'] >= confidence) & (rules['support'] >= support) ].sort_values(by=["support", "confidence", "lift"], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,(7670039905),(7670039995),0.138614,0.158416,0.108911,0.785714,4.959821,0.086952,3.927393
72,"(76700399, 7670039905)",(7670039995),0.138614,0.158416,0.108911,0.785714,4.959821,0.086952,3.927393
75,(7670039905),"(7670039995, 76700399)",0.138614,0.158416,0.108911,0.785714,4.959821,0.086952,3.927393
138,"(7670039912, 7670039905)",(7670039995),0.099010,0.158416,0.089109,0.900000,5.681250,0.073424,8.415842
235,"(76700399, 7670039912, 7670039905)",(7670039995),0.099010,0.158416,0.089109,0.900000,5.681250,0.073424,8.415842
241,"(7670039912, 7670039905)","(7670039995, 76700399)",0.099010,0.158416,0.089109,0.900000,5.681250,0.073424,8.415842
15,(7670039902),(7670039910),0.108911,0.148515,0.089109,0.818182,5.509091,0.072934,4.683168
41,"(76700399, 7670039902)",(7670039910),0.108911,0.148515,0.089109,0.818182,5.509091,0.072934,4.683168
45,(7670039902),"(76700399, 7670039910)",0.108911,0.148515,0.089109,0.818182,5.509091,0.072934,4.683168


**6. Avaliação do Modelo**

Qual produto deve ser recomendado para um usuário que navegou o produto **76700399**?

R.: Recomendaria o produto **7670039905**, pois trata-se de um item que *em conjunto com o produto* **76700399** aparecem com uma frequência de popularidade (support) de 10,89%, sendo a probabilidade de **76700399** estar associado com **7670039905** é de cerca de 78,57%. Além disso, a probabilidade de **76700399** estar associado com **7670039905** em função da sua própria popularidade é de cerca de 4,95.

**7. Sugestões de outras análises de associações de itens**

- Seria interessante ver as piores associações para que ações não envolvem tais recomendações;
- Avaliar os top produtos mais populares, a fim de elaborar uma estratégia de recomendação diferente do market basket, e comparando a eficácia de ambos;
- Elencar os top produtos com menor popularidade, a fim de entender os motivos. Avaliar as métricas (support, confidence e lift) que sustentam essa baixa popularidade para os itens em questão, verificando se estão abaixo do limiar esperado.